# Analysis of Deecntralised Iterative Auction Task Mutation

In [1]:
from __future__ import annotations

import os
import sys
sys.path.append(os.path.join(os.getcwd(), ".."))

import json
from typing import List

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from src.extra.io import save_plot

matplotlib.rcParams['font.family'] = "monospace"
%matplotlib inline

## Random Mutation

In [2]:
results_filename = '../data/task_mutation_paper_r_t30_s6_dt07-17_23-19-03.json'

with open(results_filename) as file:
    results_data = json.load(file)

In [3]:
def allocated(was_allocated, is_allocated):
    return sum(result["task allocated"] == was_allocated and result["mutant task allocated"] == is_allocated 
               for mutation_results in results_data for name, result in mutation_results.items() 
               if name != "no mutation" and name != "model")

print(f'Was allocated | Is Allocated | total ')
print(f'    True      |     True     |   {allocated(True, True)}')
print(f'    False     |     True     |   {allocated(False, True)}')
print(f'    True      |     False    |   {allocated(True, False)}')
print(f'    False     |     False    |   {allocated(False, False)}')

Was allocated | Is Allocated | total 
    True      |     True     |   0
    False     |     True     |   0
    True      |     False    |   198
    False     |     False    |   87


## Value only Mutation

In [ ]:
print(f'Was allocated | )

In [6]:
for pos, mutation_results in enumerate(results_data):
    for name, result in mutation_results.items():
        if name != "no mutation" and name != "model":
            if result['mutant price'] < result['task price'] and result['mutant task allocated'] == True:
                print(f'Result: {pos}, Mutant task name: {result["mutant task name"]}, '
                      f'Original price: {result["task price"]}, Mutant task price: {result["mutant price"]}')

In [7]:
was_allocated_results = [
    result 
    for mutation_results in results_data for name, result in mutation_results.items() 
    if name != "no mutation" and name != "model" and result["task allocated"]
]